In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
data = pd.read_csv('c:/chai.coding/python/tmdb_5000/tmdb_5000_movies.csv')

In [4]:
m = data['vote_count'].quantile(0.9)
data = data.loc[data['vote_count'] >= m]

In [5]:
C = data['vote_average'].mean()

In [6]:
print(C)
print(m)

6.9629937629937615
1838.4000000000015


In [7]:
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    
    return ( v / (v+m) * R ) + (m / (m + v) * C)

In [8]:
data['score'] = data.apply(weighted_rating, axis = 1)
data.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,score
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,7.168053
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,6.918271
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,6.493333
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,7.492998
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,6.500396


In [9]:
data[['genres', 'keywords']].head(2)

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na..."


In [10]:
data['genres'] = data['genres'].apply(literal_eval)
data['keywords'] = data['keywords'].apply(literal_eval)

In [11]:
data[['genres', 'keywords']].head(2)

,genres,keywords
0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...","[{'id': 1463, 'name': 'culture clash'}, {'id':..."
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'id': 270, 'name': 'ocean'}, {'id': 726, 'na..."


In [12]:
data['genres'] = data['genres'].apply(lambda x : [d['name'] for d in x]).apply(lambda x : " ".join(x))
data['keywords'] = data['keywords'].apply(lambda x : [d['name'] for d in x]).apply(lambda x : " ".join(x))

In [14]:
data.to_csv('c:/chai.coding/python/tmdb_5000/pre_tmdb_5000_movies.csv', index = False)

In [15]:
data.genres.head(2)

0    Action Adventure Fantasy Science Fiction
1                    Adventure Fantasy Action
Name: genres, dtype: object

In [18]:
count_vector = CountVectorizer(ngram_range=(1, 3))
c_vector_genres = count_vector.fit_transform(data['genres'])
c_vector_genres.shape

(481, 364)

In [21]:
gerne_c_sim = cosine_similarity(c_vector_genres, c_vector_genres).argsort()[:, ::-1]
gerne_c_sim.shape

(481, 481)

In [22]:
print(gerne_c_sim)

[[  0  13  42 ... 298 297 240]
 [ 11   1 200 ... 329 330 240]
 [  2 376 216 ... 314 304 240]
 ...
 [478 187  12 ... 326 327   0]
 [479 466 383 ... 220 224   0]
 [480 468 294 ... 246 248   0]]


In [23]:
def get_recommend_movie_list(df, movie_title, top=30):
    # 특정 영화와 비슷한 영화를 추천해야 하기 때문에 '특정 영화' 정보를 뽑아낸다.
    target_movie_index = df[df['title'] == movie_title].index.values
    
    #코사인 유사도 중 비슷한 코사인 유사도를 가진 정보를 뽑아낸다.
    sim_index = gerne_c_sim[target_movie_index, :top].reshape(-1)
    #본인을 제외
    sim_index = sim_index[sim_index != target_movie_index]

    #data frame으로 만들고 vote_count으로 정렬한 뒤 return
    result = df.iloc[sim_index].sort_values('score', ascending=False)[:10]
    return result

In [43]:
get_recommend_movie_list(data, movie_title = 'Inception')

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,score
66,175000000,Animation Comedy Family Adventure,http://disney.go.com/disneypictures/up/,14160,age difference central and south america ballo...,en,Up,Carl Fredricksen spent his entire life dreamin...,92.201962,"[{""name"": ""Pixar Animation Studios"", ""id"": 3}]",...,2009-05-13,735099082,96.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,Up,7.7,6870,7.544413
1541,30000000,Animation Comedy Family,http://toystory.disney.com/toy-story,862,jealousy toy boy friendship friends rivalry bo...,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",73.640445,"[{""name"": ""Pixar Animation Studios"", ""id"": 3}]",...,1995-10-30,373554033,81.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,Toy Story,7.7,5269,7.509366
42,200000000,Animation Family Comedy,http://disney.go.com/toystory/,10193,hostage college toy barbie animation escape da...,en,Toy Story 3,"Woody, Buzz, and the rest of Andy's toys haven...",59.995418,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,2010-06-16,1066969703,103.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,No toy gets left behind.,Toy Story 3,7.6,4597,7.418026
231,115000000,Animation Comedy Family,http://movies.disney.com/monsters-inc,585,monster infant energy supply company rivalry h...,en,"Monsters, Inc.","James Sullivan and Mike Wazowski are monsters,...",106.815545,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,2001-11-01,562816256,92.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,We Scare Because We Care.,"Monsters, Inc.",7.5,5996,7.373988
160,145000000,Fantasy Action Adventure Animation Comedy Family,http://www.howtotrainyourdragon.com/,82702,father son relationship wife husband relations...,en,How to Train Your Dragon 2,The thrilling second chapter of the epic How T...,100.213910,"[{""name"": ""DreamWorks Animation"", ""id"": 521}, ...",...,2014-06-12,609123048,102.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The training is over.,How to Train Your Dragon 2,7.6,3106,7.363152
744,60000000,Adventure Animation Comedy Family Fantasy,http://www.thelegomovie.com,137106,father son relationship creativity friendship ...,en,The Lego Movie,"An ordinary Lego mini-figure, mistakenly thoug...",59.547928,"[{""name"": ""Village Roadshow Pictures"", ""id"": 7...",...,2014-02-06,469160692,100.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The story of a nobody who saved everybody.,The Lego Movie,7.5,3070,7.298869
899,60000000,Adventure Animation Comedy Family Fantasy,http://www.shrek.com/,808,magic liberation lordship castle robin hood en...,en,Shrek,It ain't easy bein' green -- especially if you...,67.298732,"[{""name"": ""DreamWorks SKG"", ""id"": 27}, {""name""...",...,2001-05-16,484409218,90.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The greatest fairy tale never told.,Shrek,7.3,4056,7.194891
343,90000000,Animation Comedy Family,http://toystory.disney.com/toy-story-2,863,museum prosecution identity crisis airplane fl...,en,Toy Story 2,"Andy heads off to Cowboy Camp, leaving his toy...",73.575118,"[{""name"": ""Pixar Animation Studios"", ""id"": 3}]",...,1999-10-30,497366869,92.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The toys are back!,Toy Story 2,7.3,3806,7.190236
89,165000000,Family Animation Comedy Adventure,http://disney.go.com/wreck-it-ralph,82690,support group product placement bullying racin...,en,Wreck-It Ralph,"Wreck-It Ralph is the 9-foot-tall, 643-pound v...",62.341073,"[{""name"": ""Walt Disney Animation Studios"", ""id...",...,2012-11-01,471222889,108.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The story of a regular guy just looking for a ...,Wreck-It Ralph,7.1,4570,7.060697
631,59000000,Animati

In [25]:
data[data['title'] == 'The Dark Knight Rises']

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,score
3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.31295,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,7.492998
